<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpustools/blob/main/S101lemKA_unknown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing pipeline for DAAD project
## Stage 1: Lemmatization (Georgian)

# Georgian 'random' corpus 
- (collected in SketchEngine using non-topical high-frequent keywords from the top 1000 of the wikipedia list)

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/362c24e311104c9aa35b/?dl=1
!mv index.html?dl=1 georgianrandom02.txt

In [ ]:
%%bash
# installing TreeTagger (en, de, ka)
mkdir treetagger
cd treetagger
# Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# Download the tagging scripts into the same directory.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
gunzip tagger-scripts.tar.gz
# Download the installation script install-tagger.sh.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# Download the parameter files for the languages you want to process.
# list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/english.par.gz
sh install-tagger.sh
cd ..
sudo pip install treetaggerwrapper
# changing options: no-unknown, sgml, lemma
mv /content/treetagger/cmd/tree-tagger-english /content/tree-tagger-english0
awk '{ if (NR == 9) print "OPTIONS=\"-token -lemma -sgml -no-unknown\""; else print $0}' /content/tree-tagger-english0 > /content/treetagger/cmd/tree-tagger-english
chmod a+x ./treetagger/cmd/tree-tagger-english
# downloading German and Georgian 
wget https://heibox.uni-heidelberg.de/f/ec8226edebb64a359407/?dl=1
mv index.html?dl=1 /content/treetagger/lib/german-utf8.par
wget https://heibox.uni-heidelberg.de/f/9183090d2bdb41e09055/?dl=1
mv index.html?dl=1 /content/treetagger/lib/georgian.par
wget https://heibox.uni-heidelberg.de/f/9cafab0509d64ed1ac4b/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-georgian2
cp /content/treetagger/cmd/tree-tagger-georgian2 /content/treetagger/cmd/tree-tagger-georgian
# German2 = -no-unknown 
# note: tree-tagger-german will not work, as parameter files have not been downloaded, only use tree-tagger-german2 with utf8 encoding
wget https://heibox.uni-heidelberg.de/f/acb9b8a2fa4f40e08f8a/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-german2
chmod a+x /content/treetagger/cmd/tree-tagger-georgian2
chmod a+x /content/treetagger/cmd/tree-tagger-german2

In [ ]:
%%bash
wget https://heibox.uni-heidelberg.de/f/a6f7f36f175942ccad0a/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-georgian
chmod a+x /content/treetagger/cmd/tree-tagger-georgian

# cp /content/treetagger/cmd/tree-tagger-georgian2 /content/treetagger/cmd/tree-tagger-georgian


In [4]:
# analysing German file 
# !./treetagger/cmd/tree-tagger-german2 /content/udhr/udhr_kat_v02.txt >/content/udhrTT/udhr_kat_vert.txt
!./treetagger/cmd/tree-tagger-georgian georgianrandom02.txt >georgianrandom02.vert

	reading parameters ...
	tagging ...
11301000	 finished.


In [ ]:
!head -n 50 georgianrandom02.vert


In [6]:
!wc georgianrandom02.vert

 11301824  33112275 394150707 georgianrandom02.vert


In [7]:
## how to print counter in AWK:
## https://stackoverflow.com/questions/67901330/awk-how-to-print-the-last-value-of-a-counter

# !awk -F '\t' '(NF==3){printf "%s ", $3; if(FNR % 10000 == 0){printf "\n"}}' < /content/udhrTT/udhr_deu_1996_vert.txt >/content/udhrTT/udhr_deu_1996_lem.txt

# if we need <p> tags for paragraphs:
# !awk -F '\t' '(NF==3){printf "%s ", $3}(NF!=3){printf "\n%s\n", $0}' < /content/udhrTT/udhr_deu_1996_vert.txt >/content/udhrTT/udhr_deu_1996_lem.txt
# if we do not need paragraph tags
# !awk -F '\t' '(NF==3){printf "%s ", $3}(NF!=3){printf "\n"}' < /content/udhrTT/udhr_kat_vert.txt >/content/udhrTT/udhr_kat_lem.txt
!awk -F '\t' '($3=="<unknown>"){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' <georgianrandom02.vert >georgianrandom02-unknown.txt

2123093


In [ ]:
!head -n 50 georgianrandom02-unknown.txt


In [ ]:
!tail -n 20 georgianrandom02-unknown.txt

In [10]:
D = {}
with open("georgianrandom02-unknown.txt", 'r') as f:
    for line in f:
        line = line.rstrip()
        try:
            D[line] +=1
        except:
            D[line] = 1

fo = open("georgianrandom02-unknown-frq.txt", 'w')
ICorpusLen = 11301824
IUnknown = 2123093
FCoverage = IUnknown/ICorpusLen
print('percent Unknown TT: ', FCoverage)
IRestUnknown = IUnknown
c = 0
for key, val in sorted(D.items(), key=lambda item: item[1], reverse=True):
    c+=1
    IRestUnknown = IRestUnknown - val
    FRestCoverage = IRestUnknown/ICorpusLen
    fo.write(str(c) + '\t' + key + '\t' + str(val) + '\t' + str(FRestCoverage) + '\n')
    

percent Unknown TT:  0.18785401365301743


In [ ]:
!head -n 50 georgianrandom02-unknown-frq.txt

## GIP DAAD corpus
### checking coverage

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/1981a35452db40c583c0/?dl=1
!mv index.html?dl=1 select-kat.tgz

In [ ]:
!tar xvzf select-kat.tgz

In [ ]:
import os
FOut = open('georgian-corp-gip-v01.txt', 'w')
for root, dirs, files in os.walk("./select-kat"):
    path = root.split(os.sep)
    print((len(path) - 1) * '---', os.path.basename(root))
    for file in files:
        print(len(path) * '---', file)
        FOut.write('<doc filename="' + file + '">\n')
        SFilePath = os.path.join(root,file)
        FIn = open(SFilePath, 'r')
        SIn = FIn.read()
        FOut.write(SIn)
        FOut.write('</doc>\n\n')

FOut.flush()
FOut.close()



In [27]:
!wc georgian-corp-gip-v01.txt

   44448   892632 19802190 georgian-corp-gip-v01.txt


In [23]:
!./treetagger/cmd/tree-tagger-georgian georgian-corp-gip-v01.txt >georgian-corp-gip-v01.vert

	reading parameters ...
	tagging ...
1086000	 finished.


In [26]:
!wc georgian-corp-gip-v01.vert

 1086727  3260172 40704147 georgian-corp-gip-v01.vert


In [24]:
!awk -F '\t' '($3=="<unknown>"){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' <georgian-corp-gip-v01.vert >georgian-corp-gip-v01-unknown.txt

222297


In [28]:
ICorpusLen = 1086727
IUnknown = 222297
FCoverage = IUnknown/ICorpusLen
print('percent Unknown TT: ', FCoverage)

percent Unknown TT:  0.2045564341366323


In [25]:
E = {}
with open("georgian-corp-gip-v01-unknown.txt", 'r') as f:
    for line in f:
        line = line.rstrip()
        try:
            E[line] +=1
        except:
            E[line] = 1

In [32]:
## checking coverage on georgianrandom02 corpus
IRestUnknown = IUnknown
c = 0
CFound = 0
fo = open("georgian-corp-gip-v01-unknown-from-georgianrandom02-frq.txt", 'w')
for key, val in sorted(D.items(), key=lambda item: item[1], reverse=True):
    c+=1
    BFound = False
    try: 
        val2 = E[key]
        BFount = True
        CFound += 1
    except:
        val2 = 0

    IRestUnknown = IRestUnknown - val2
    FRestCoverage = IRestUnknown/ICorpusLen
    fo.write(str(c) + '\t' + key + '\t' + str(val) + '\t' + str(BFound) + '\t' + str(CFound) + '\t' + str(FRestCoverage) + '\n')



# Optional dataset (not used)
## Example: Universal declaration of human rights (

In [ ]:
# Downloading UDHR
!wget https://unicode.org/udhr/assemblies/udhr_txt.zip


In [ ]:
%%bash
mkdir udhr
mv udhr_txt.zip udhr
cd udhr/
unzip udhr_txt.zip

In [ ]:
%%bash
# put paragraph tags
awk '{print "<p>\n"$0 ; print "</p>"}' /content/udhr/udhr_kat.txt >/content/udhr/udhr_kat_v02.txt